In [1]:
import os

os.environ['WANDB_API_KEY'] = ''

In [2]:
import pip

def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])
        
install('wandb')

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable

Requirement already satisfied: wandb in /home/ubuntu/.local/lib/python3.10/site-packages (0.16.0)

Requirement already satisfied: protobuf!=4.21.0,<5,>=3.19.0 in /home/ubuntu/.local/lib/python3.10/site-packages (from wandb) (4.25.1)

Requirement already satisfied: setuptools in /usr/lib/python3/dist-packages (from wandb) (59.6.0)

...

Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.10/dist-packages (from requests<3,>=2.0.0->wandb) (3.4)

Requirement already satisfied: certifi>=2017.4.17 in /home/ubuntu/.local/lib/python3.10/site-packages (from requests<3,>=2.0.0->wandb) (2023.11.17)

Requirement already satisfied: urllib3<3,>=1.21.1 in /home/ubuntu/.local/lib/python3.10/site-packages (from requests<3,>=2.0.0->wandb) (2.1.0)

Requirement already satisfied: charset-normalizer<4,>=2 in /home/ubuntu/.local/lib/python3.10/site-packages (from requests<3,>=2.0.0->wandb) (3.3.2)

Requirement already satisfied: smmap<6,>=3.0.1 in /home/ubuntu/.local/lib/python3.10/site-packages (from gitdb<5,>=4.0.1->GitPython!=3.1.29,>=1.0.0->wandb) (5.0.1)

In [3]:
from transformers import (
    AutoConfig, 
    AutoModelForCausalLM, 
    AutoTokenizer, 
    default_data_collator,
    TrainingArguments,
    Trainer,
    TrainerCallback
)
from datasets import load_dataset, load_from_disk

import ray
from ray import train
from ray.train import Checkpoint
from ray.train.huggingface import TransformersTrainer
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer, get_device
from transformers.trainer_utils import get_last_checkpoint
import torch
from torch.utils.data import DataLoader
import numpy as np
import os
import shutil
import wandb

wandb_api = wandb.Api()

In [4]:
WANDB_PROJECT = os.environ.get('WANDB_PROJECT', 'run-ray')
WANDB_API_KEY = os.environ.get('WANDB_API_KEY', wandb_api.api_key)
MINIO = os.environ.get('MINIO', 'http://minio:9000')

In [5]:
def train_func(config):
    
    from streaming.base.format.mds.encodings import Encoding, _encodings
    from streaming import StreamingDataset
    import streaming
    import s3fs
    
    MINIO = os.environ.get('MINIO', 'http://minio:9000')
    fs = s3fs.S3FileSystem(endpoint_url = MINIO, anon = True)
    
    device = str(get_device()).replace(':', '-')
    print(device)

    class UInt16(Encoding):
        def encode(self, obj) -> bytes:
            return obj.tobytes()

        def decode(self, data: bytes):
            return np.frombuffer(data, np.uint16)

    _encodings['uint16'] = UInt16
    
    class DatasetFixed(torch.utils.data.Dataset):
        def __init__(self, local, remote):
            
            streaming.base.util.clean_stale_shared_memory()
            self.dataset = StreamingDataset(local=local, remote = remote, download_timeout = 300)

        def __getitem__(self, idx):
            data = self.dataset[idx]
            data['labels'] = data["input_ids"].copy()
            data.pop('token_type_ids', None)
            for k in data.keys():
                data[k] = data[k].astype(np.int64)
            return data

        def __len__(self):
            return len(self.dataset)
    
    local = os.path.join('/home/ubuntu/storage', config['local'])
    shutil.rmtree(local, ignore_errors = True)
    train_dataset = DatasetFixed(local=local, remote=config['remote'])
    
    # https://github.com/mosaicml/streaming/issues/307#issuecomment-1729829065
    def inf_loop_dataloader(dataloader: torch.utils.data.DataLoader):
        while True:
            for batch in dataloader:
                yield batch
    dataloader = DataLoader(train_dataset, batch_size=2)
    dataset_iterator = iter(inf_loop_dataloader(dataloader))
    batch = next(iter(dataset_iterator))

#     train_dataset = load_from_disk('s3://train/wiki-test', 
#                                  storage_options=fs.storage_options, 
#                                  keep_in_memory = False)
    
    MODEL_NAME = "gpt2"
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
    directory = f'{MODEL_NAME}-test'
    
    output_dir = os.path.join('/home/ubuntu/storage', directory)
    output_temp = os.path.join('/home/ubuntu/storage', 'temp' + device)
    s3_bucket = 'train'
    s3_output_dir = os.path.join(s3_bucket, directory)
    
    args = TrainingArguments(
        output_dir=output_dir,
        save_strategy="steps",
        logging_strategy="steps",
        learning_rate=2e-5,
        weight_decay=0.01,
        max_steps=10000,
        save_steps = 100,
        save_total_limit = 2,
        logging_steps = 1,
        per_device_train_batch_size = 6,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        data_collator=default_data_collator,
    )
    
    
    class S3Callback(TrainerCallback):
        def on_save(self, args, state, control, **kwargs):
            fs.delete(s3_output_dir)
            fs.put(output_dir, s3_bucket, recursive=True)
        
    trainer.add_callback(S3Callback())
    
    checkpoints = []
    try:
        checkpoints = fs.ls(s3_output_dir)
        checkpoints = [f for f in checkpoints if 'checkpoint-' in f]
        checkpoints = sorted(checkpoints, key = lambda x: int(x.split('-')[-1]))
    except:
        pass
    
    if len(checkpoints):
        checkpoint = checkpoints[-1]
        print(f'load checkpoint from {checkpoint} into {output_temp}')
        shutil.rmtree(output_temp, ignore_errors = True)
        fs.get(checkpoint, output_temp, recursive = True)
        trainer.train(resume_from_checkpoint=output_temp)
    else:
        trainer.train()

In [6]:
runtime_env = {
    'pip': ['wandb', 's3fs', 'mosaicml-streaming'],
    'env_vars': {
        'WANDB_PROJECT': WANDB_PROJECT,
        'WANDB_API_KEY': WANDB_API_KEY,
        'S3_ENDPOINT_URL': MINIO,
        'MINIO': MINIO,
    }
}

In [7]:
class RayConnection:
    def __init__(self, address, **kwargs):
        ray.init(address=address, **kwargs)

    def __enter__(self):
        return self

    def __exit__(self, typ, value, traceback):
        ray.shutdown()

In [ ]:
with RayConnection("ray://localhost:10001", runtime_env=runtime_env):
    scaling_config = ScalingConfig(
        num_workers=8, 
        use_gpu=True,
    )
    run_config = train.RunConfig(failure_config=train.FailureConfig(max_failures=-1))
    ray_trainer = TorchTrainer(
        train_func,
        train_loop_config={
            'local': 'local_dir',
            'remote': 's3://train/indexed'
        },
        scaling_config=scaling_config,
        run_config=run_config

    )
    result = ray_trainer.fit()

(TunerInternal pid=33311) [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


(TunerInternal pid=33311) 
(TunerInternal pid=33311) View detailed results here: /home/ubuntu/ray_results/TorchTrainer_2023-11-22_17-57-28
(TunerInternal pid=33311) To visualize your results with TensorBoard, run: `tensorboard --logdir /home/ubuntu/ray_results/TorchTrainer_2023-11-22_17-57-28`


(TunerInternal pid=33311) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


(TunerInternal pid=33311) 
(TunerInternal pid=33311) Training started with configuration:
(TunerInternal pid=33311) ╭───────────────────────────────────────────────╮
(TunerInternal pid=33311) │ Training config                               │
(TunerInternal pid=33311) ├───────────────────────────────────────────────┤
(TunerInternal pid=33311) │ train_loop_config/local             local_dir │
(TunerInternal pid=33311) │ train_loop_config/remote   s3://train/indexed │
(TunerInternal pid=33311) ╰───────────────────────────────────────────────╯


(TorchTrainer pid=33520) Starting distributed worker processes: ['33620 (10.208.0.249)', '33621 (10.208.0.249)', '33625 (10.208.0.249)', '33626 (10.208.0.249)', '3487 (10.208.0.238)', '3488 (10.208.0.238)', '3489 (10.208.0.238)', '3490 (10.208.0.238)']
(RayTrainWorker pid=33620) Setting up process group for: env:// [rank=0, world_size=8]


...

  5%|▌         | 549/10000 [00:37<1:11:12,  2.21it/s]


(RayTrainWorker pid=33620) {'loss': 7.2892, 'learning_rate': 1.8902000000000003e-05, 'epoch': 0.84}
(RayTrainWorker pid=3487, ip=10.208.0.238) {'loss': 7.2892, 'learning_rate': 1.8902000000000003e-05, 'epoch': 0.84}
(RayTrainWorker pid=3487, ip=10.208.0.238) {'loss': 7.1592, 'learning_rate': 1.8900000000000002e-05, 'epoch': 0.85}
(RayTrainWorker pid=33620) {'loss': 7.1592, 'learning_rate': 1.8900000000000002e-05, 'epoch': 0.85}


  6%|▌         | 550/10000 [00:36<18:03,  8.73it/s]


(RayTrainWorker pid=3487, ip=10.208.0.238) {'loss': 7.1803, 'learning_rate': 1.8898e-05, 'epoch': 0.85}
(RayTrainWorker pid=33620) {'loss': 7.1803, 'learning_rate': 1.8898e-05, 'epoch': 0.85}


  6%|▌         | 551/10000 [00:37<18:02,  8.73it/s]
